In [1]:
import pandas as pd
import numpy as np

In [2]:
def update_produv(produv_train):
    
    produv_start=pd.pivot_table(produv_train,index='NPLV',values='SEC', aggfunc={'min'}).reset_index()
    produv_end=pd.pivot_table(produv_train,index='NPLV',values='SEC', aggfunc={'max'}).reset_index()
    ras_med=pd.pivot_table(produv_train,index='NPLV',values='RAS', aggfunc={'mean'}).reset_index()
    ras_sum=pd.pivot_table(produv_train,index='NPLV',values='RAS', aggfunc={'sum'}).reset_index()
    pol_start=pd.pivot_table(produv_train,index='NPLV',values='POL', aggfunc={'max'}).reset_index()
    pol_end=pd.pivot_table(produv_train,index='NPLV',values='POL', aggfunc={'min'}).reset_index()
    
    produv_upd=pd.merge(produv_start,produv_end, how='left',on='NPLV')
    produv_upd=pd.merge(produv_upd,ras_sum, how='left',on='NPLV')
    produv_upd=pd.merge(produv_upd,pol_start, how='left',on='NPLV')
    produv_upd=pd.merge(produv_upd,pol_end, how='left',on='NPLV')
    
    #df.rename(columns = {list(df)[1]:'new_name'}, inplace=True)
    
    produv_upd.rename(columns={list(produv_upd)[1]: 'PRODUVKA_START'}, inplace=True)
    produv_upd.rename(columns={list(produv_upd)[2]: 'PRODUVKA_END'}, inplace=True)
    produv_upd.rename(columns={list(produv_upd)[3]: 'RASHOD_SUM'}, inplace=True)
    produv_upd.rename(columns={list(produv_upd)[4]: 'POL_MAX'}, inplace=True)
    produv_upd.rename(columns={list(produv_upd)[5]: 'POL_MIN'}, inplace=True)
    
    produv_upd['PRODUVKA_TIME']=produv_upd['PRODUVKA_END']-produv_upd['PRODUVKA_START']
    produv_upd['PRODUVKA_TIME,SEC']=produv_upd['PRODUVKA_TIME'].dt.seconds

    
    return produv_upd

In [3]:
path='D:/data/'

names=['chronom','chugun','gas','lom','plavki','produv','sip']

chrom_train=pd.read_csv(path+names[0]+'_train.csv', )
chugun_train=pd.read_csv(path+names[1]+'_train.csv')
gas_train=pd.read_csv(path+names[2]+'_train.csv')
lom_train=pd.read_csv(path+names[3]+'_train.csv')
plavki_train=pd.read_csv(path+names[4]+'_train.csv')
produv_train=pd.read_csv(path+names[5]+'_train.csv')
sip_train=pd.read_csv(path+names[6]+'_train.csv')

chrom_test=pd.read_csv(path+names[0]+'_test.csv')
chugun_test=pd.read_csv(path+names[1]+'_test.csv')
gas_test=pd.read_csv(path+names[2]+'_test.csv')
lom_test=pd.read_csv(path+names[3]+'_test.csv')
plavki_test=pd.read_csv(path+names[4]+'_test.csv')
produv_test=pd.read_csv(path+names[5]+'_test.csv')
sip_test=pd.read_csv(path+names[6]+'_test.csv')

target_train=pd.read_csv(path+'target_train.csv')
   

In [4]:
chrom_train.head()

,Unnamed: 0,NPLV,TYPE_OPER,NOP,VR_NACH,VR_KON,O2
0,35171,510008,межпл.прост.,межпл. простой,2011-01-01 10:28:13,2011-01-01 10:32:15,NaN
1,35162,510008,межпл.прост.,межпл. простой,2021-01-01 03:01:07,2021-01-01 03:08:11,NaN
2,35163,510008,межпл.прост.,Осмотр конвертера,2021-01-01 03:01:19,2021-01-01 03:03:43,NaN
3,35164,510008,межпл.прост.,Наведение гарнисажа,2021-01-01 03:03:43,2021-01-01 03:05:23,NaN
4,35165,510008,межпл.прост.,Слив шлака,2021-01-01 03:05:23,2021-01-01 03:06:09,NaN


In [5]:
chrom_train=chrom_train[chrom_train['VR_NACH']>'2012-01-01 10:28:13']

In [6]:
chrom_train

,Unnamed: 0,NPLV,TYPE_OPER,NOP,VR_NACH,VR_KON,O2
1,35162,510008,межпл.прост.,межпл. простой,2021-01-01 03:01:07,2021-01-01 03:08:11,NaN
2,35163,510008,межпл.прост.,Осмотр конвертера,2021-01-01 03:01:19,2021-01-01 03:03:43,NaN
3,35164,510008,межпл.прост.,Наведение гарнисажа,2021-01-01 03:03:43,2021-01-01 03:05:23,NaN
4,35165,510008,межпл.прост.,Слив шлака,2021-01-01 03:05:23,2021-01-01 03:06:09,NaN
5,35166,510008,межпл.прост.,Отсутствие O2,2021-01-01 03:06:09,2021-01-01 03:08:15,NaN
...,...,...,...,...,...,...,...
34401,20943,512322,опер,Повалка,2021-04-26 18:31:26,2021-04-26 18:41:10,NaN
34402,20944,512322,опер,Выпуск металла,2021-04-26 18:41:10,2021-04-26 18:47:40,NaN
34403,20945,512322,опер,Отсечка шлака,2021-04-26 18:46:15,2021-04-26 18:46:15,NaN
34404,20946,512322,опер,Прерыв. выпуска,2021-04-26 18:47:40,2021-04-26 18:48:40,NaN


In [7]:
produv_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4729802 entries, 0 to 4729801
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   NPLV    int64  
 1   SEC     object 
 2   RAS     float64
 3   POL     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 144.3+ MB


In [8]:
produv_train.head(10)

,NPLV,SEC,RAS,POL
0,510008,2021-01-01 03:18:26,382.00,3.92
1,510008,2021-01-01 03:18:28,382.00,3.92
2,510008,2021-01-01 03:18:30,553.00,3.92
3,510008,2021-01-01 03:18:32,701.00,3.92
4,510008,2021-01-01 03:18:34,813.00,3.92
5,510008,2021-01-01 03:18:36,888.00,3.92
6,510008,2021-01-01 03:18:38,939.00,3.45
7,510008,2021-01-01 03:18:40,965.00,2.87
8,510008,2021-01-01 03:18:42,962.45,2.87
9,510008,2021-01-01 03:18:44,959.90,2.87


In [9]:
produv_train['SEC']=pd.to_datetime(produv_train['SEC'], format='%Y-%m-%d %H:%M:%S')

In [10]:
produv_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4729802 entries, 0 to 4729801
Data columns (total 4 columns):
 #   Column  Dtype         
---  ------  -----         
 0   NPLV    int64         
 1   SEC     datetime64[ns]
 2   RAS     float64       
 3   POL     float64       
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 144.3 MB


In [11]:
produv_upd=update_produv(produv_train)

In [12]:
produv_upd

,NPLV,PRODUVKA_START,PRODUVKA_END,RASHOD_SUM,POL_MAX,POL_MIN,PRODUVKA_TIME,"PRODUVKA_TIME,SEC"
0,510008,2021-01-01 03:18:26,2021-01-01 04:26:54,838814.0,7.070000,0.77,0 days 01:08:28,4108
1,510009,2021-01-01 04:26:56,2021-01-01 05:25:08,764750.0,4.760000,0.63,0 days 00:58:12,3492
2,510010,2021-01-01 05:25:10,2021-01-01 06:23:50,764170.5,4.349765,0.65,0 days 00:58:40,3520
3,510011,2021-01-01 06:23:52,2021-01-01 07:26:04,733717.0,12.490000,0.79,0 days 01:02:12,3732
4,510012,2021-01-01 07:26:06,2021-01-01 08:36:44,840471.0,5.000000,0.67,0 days 01:10:38,4238
...,...,...,...,...,...,...,...,...
2058,512318,2021-04-26 13:10:12,2021-04-26 14:37:38,978520.0,10.100000,0.80,0 days 01:27:26,5246
2059,512319,2021-04-26 14:37:40,2021-04-26 15:35:46,783447.0,10.040000,0.83,0 days 00:58:06,3486
2060,512320,2021-04-26 15:35:48,2021-04-26 16:36:02,742971.0,7.270000,0.90,0 days 01:00:14,3614
2061,512321,2021-04-26 16:36:04,2021-04-26 18:08:46,964755.5,10.580000,0.92,0 days 01:32:42,5562


In [16]:
gas_train.head(100)

,NPLV,Time,V,T,O2,N2,H2,CO2,CO,AR,T фурмы 1,T фурмы 2,O2_pressure
0,510008,2021-01-01 03:08:11.437,218263.343750,262.847229,18.722993,80.132247,0.087755,0.163878,0.009229,0.893243,0.0,0.0,13.085938
1,510008,2021-01-01 03:08:12.437,218263.343750,262.847229,18.732721,80.138406,0.087959,0.148980,0.008390,0.892948,0.0,0.0,13.085938
2,510008,2021-01-01 03:08:13.437,218369.359375,262.152771,18.742449,80.144565,0.088163,0.134082,0.007551,0.892653,0.0,0.0,13.085938
3,510008,2021-01-01 03:08:14.437,218475.359375,261.805573,18.752177,80.150724,0.088367,0.119184,0.006712,0.892358,0.0,0.0,13.093172
4,510008,2021-01-01 03:08:15.437,218369.359375,260.763885,18.761905,80.156883,0.088571,0.104286,0.005873,0.892063,0.0,0.0,13.093172
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,510008,2021-01-01 03:09:47.437,211257.265625,230.902771,29.490000,68.599998,0.000000,0.790000,0.240000,0.880000,0.0,0.0,13.049767
96,510008,2021-01-01 03:09:48.437,210378.843750,231.597214,29.670000,68.430000,0.000000,0.810000,0.210000,0.870000,0.0,0.0,13.035300
97,510008,2021-01-01 03:09:49.437,210708.687500,232.638901,29.760000,68.250000,0.000000,0.850000,0.270000,0.880000,0.0,0.0,13.049767
98,510008,2021-01-01 03:09:50.437,210708.687500,232.638901,29.760000,68.250000,0.000000,0.850000,0.270000,0.880000,0.0,0.0,13.049767


In [14]:
lom_train.head()

,NPLV,VDL,NML,VES
0,510008,4,К,56500
1,510008,8,О,16700
2,510008,13,КП,3000
3,510009,4,К,49800
4,510009,8,О,22800


In [15]:
sip_train.head()

,NPLV,VDSYP,NMSYP,VSSYP,DAT_OTD
0,510008,346,Уголь ТО,570,2021-01-01 03:03:53
1,510008,346,Уголь ТО,220,2021-01-01 03:04:10
2,510008,408,изв_ЦОИ,7300,2021-01-01 03:08:17
3,510008,346,Уголь ТО,270,2021-01-01 03:09:03
4,510008,346,Уголь ТО,430,2021-01-01 03:09:20
